## pip install

In [ ]:
!pip install konlpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 25.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 465.3/465.3 kB 30.0 MB/s eta 0:00:00


## 상수 설정

In [ ]:
EVALUATION_DATASET_PATH = './drive/MyDrive/Models/eval_queries.jsonl'
EVALUATED_INFERENCE_PATH = './drive/MyDrive/Models/eval_response_kogpt2F.txt'

## import

In [ ]:
import numpy as np
from collections import Counter
from nltk import ngrams
import json

from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
with open(EVALUATION_DATASET_PATH, 'r') as r:
  eval_queries = json.load(r)
  references = [query['references'] for query in eval_queries]

with open(EVALUATED_INFERENCE_PATH, 'r') as rf:
	candidates = rf.read().split('\n')


print(candidates[:5])
print(references[:5])

['응, 맞아. 그런 느낌이야. 너도 그런 특수 시술을 받고 싶어해?', '그래서 조금은 지칠 때가 있잖아. 어떻게 하면 좀 쉬어질까?', '사랑한다는 의지와 의지가 있으시군요! 어떤 상황에서 그런 인연을', '아마 도움이 필요하면 같이 할 것 같아.', '그랬군요. 그러면 나중에 다른 영화 추천해주세요!']
[['어쨌든 괜찮을 거 같은데? 길게 안 하면', '무슨 특수 펌인지 잘 모르겠어. 설명해줄 수 있을까?', '그러게. 처음 보는 스타일이라서 뭔지 잘 모르겠어.'], ['아고 그렇겠다 진짜 ㅠㅠ 대기업인 만큼 진짜 일 많이 한다더라', '정말 힘들구나. 스트레스 때문에 그런 거라면 같이 얘기하거나 해소 방법을 찾는 게 좋을 것 같아.', '그랬구나 ㅜㅜ 잘 위로해주고 응원해줘.'], ['너랑 얘기하다 보니까 그냥 결혼 안 하고 동물들이랑 사는 게 좋은가 싶기도 하다 야...', '무슨 뜻인지 잘 이해가 안 돼. 왜 사랑해서 결혼하는 게 기를 꺾는 거야? 좀 더 설명해줄래?', '그러게 말이다. 서로 잘 지내야 하는데 말이야.'], ['가만히 있는 건 아니니까! 움직이잖아', '건강은 우리 삶에서 정말 중요한 요소야. 건강하면 더 행복하고 활기찬 삶을 살 수 있어.', '운동이 되니까 몸에 좋아.'], ['그래도 그나마 그런 플라스틱은 눈에 보이니까 구조를 할 수 있었다지만 요새는 또 미세플라스틱이 논란이잖아요', '미안하다는 말을 하고 싶은 거야? 무슨 일이 있었는지 말해줄래?', '조심할 필요가 있어. 다음부터는 미안한 일 만들지 말자.']]


# BLEU Score

## KKma with smoothing function

In [ ]:
from nltk.translate.bleu_score import corpus_bleu, sentence_bleu, SmoothingFunction
from konlpy.tag import *

kkma = Kkma()

In [ ]:
smoothing = SmoothingFunction().method1

score = 0

for i in range(len(candidates)):
    candidate = kkma.morphs(candidates[i])
    reference = [kkma.morphs(ref) for ref in references[i]]

    bleu_score_smooth = sentence_bleu(reference, candidate, smoothing_function=smoothing)
    score += bleu_score_smooth

    print(f'Smoothed BLEU score for sentence {i+1}: ', bleu_score_smooth)

print(f'Avg BLEU score : {score / len(candidates)}')

Smoothed BLEU score for sentence 1:  0.023956565612760213
Smoothed BLEU score for sentence 2:  0.012673718536830811
Smoothed BLEU score for sentence 3:  0.025281168697394947
Smoothed BLEU score for sentence 4:  0.05344445934790546
Smoothed BLEU score for sentence 5:  0.02603075873708404
Smoothed BLEU score for sentence 6:  0.041081071978771734
Smoothed BLEU score for sentence 7:  0.02101654884156187
Smoothed BLEU score for sentence 8:  0.009678976922165645
Smoothed BLEU score for sentence 9:  0.07786777655685517
Smoothed BLEU score for sentence 10:  0.02625142896527897
Smoothed BLEU score for sentence 11:  0.16731078418090023
Smoothed BLEU score for sentence 12:  0.029176300840900796
Smoothed BLEU score for sentence 13:  0.016451990399717707
Smoothed BLEU score for sentence 14:  0.016562494897332036
Smoothed BLEU score for sentence 15:  0.03440134693333791
Smoothed BLEU score for sentence 16:  0.03058760346458022
Smoothed BLEU score for sentence 17:  0.010088815311934719
Smoothed BLEU 

## BERT Score

In [ ]:
!pip install bert-score
!pip install rouge-score

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


## METEOR Score

BLEU처럼 automated evaluation 방식.
유의어를 반영할 수 없음.
unigram에서 precision, recall의 조화평균 값이므로 candidate에서 같은 단어 잘 찍기만 하면 성능 올라감.

candidate에 대응하는 reference들(3개)에 대해 각각 meteor score를 구한 후, 그 중 최댓값이 해당 candidate의 METEOR Score.

전체 candidate의 METEOR Score의 평균을 계산할 예정.

https://misconstructed.tistory.com/64

In [ ]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [ ]:
from nltk.translate import meteor_score
from konlpy.tag import Kkma
kkma = Kkma()

sum_meteor_score = 0.0

for i in range(len(candidates)):
    candidate = kkma.morphs(candidates[i])
    met_max = 0.0 

    for j in range(3):
      reference = kkma.morphs(references[i][j])
      met_score  = meteor_score.single_meteor_score(reference, candidate)

      if met_score > met_max:
          met_max = met_score
    
    sum_meteor_score += met_max

print(f"{EVALUATED_INFERENCE_PATH} 평균 METEOR 점수 : {sum_meteor_score / len(candidates)}")


./drive/MyDrive/Models/eval_response_kogpt2F.txt 평균 METEOR 점수 : 0.1779385710150873


## Rouge Score
Machine translation에서 주로 사용하는 BLEU가 n-gram Precision에 기반한 지표라면,

ROUGE는 이름 그대로 n-gram Recall에 기반하여 계산됩니다.

rougeL, rougeLSum은 순서에 영향을 받는 지표이기 때문에, 어순이 달라져도 의미가 통하는 한국어 특성 상 부정확한 지표입니다.

hugging face library 사용했습니다.
https://huggingface.co/spaces/evaluate-metric/rouge

In [ ]:
!pip install evaluate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import evaluate

rouge = evaluate.load('rouge')
results = rouge.compute(predictions=candidates, references=references)
print(f"{EVALUATED_INFERENCE_PATH} ROUGE Score : {results}")

./drive/MyDrive/Models/eval_response_kogpt2F.txt ROUGE Score : {'rouge1': 0.016666666666666666, 'rouge2': 0.0, 'rougeL': 0.016666666666666666, 'rougeLsum': 0.016666666666666666}


## BERTScore
BERT 모델을 이용해서 문장을 토크나이징함. 따라서 유의어 반영 가능!

In [ ]:
from bert_score import score
from transformers import logging
logging.set_verbosity_error()

sum_bert_score = 0.0

for i in range(len(candidates)):
  max_score = 0.0

  for j in range(3):
    P, R, F1 = score([candidates[i]], [references[i][j]], lang='ko', model_type='bert-base-multilingual-cased')
    if max_score < F1.item():
      max_score = F1.item()
  
  sum_bert_score += max_score
  

print(f"{EVALUATED_INFERENCE_PATH} BERT F1 score : {sum_bert_score / len(candidates)}")

./drive/MyDrive/Models/eval_response_kogpt2F.txt BERT F1 score : 0.7283183652162551
